## Particle cross-section studies
This notebook "automates" the procedure used to obtain particle cross-sections for electro- and photo-production at the different colliders. This works for both _ep_ and _eA_, where _A_ is the corresponding heavy-ion target at the different colliders. 
The colliders are defined as follows:
- eRHIC: 18 GeV electrons on 275 GeV protons or 100 GeV/n Gold
- JLEIC: 10 GeV electrons on 100 GeV protons or 40 GeV/n Lead
- LHeC: 60 GeV electrons  on 7 TeV protons or 2.8 TeV/n Lead
- CEBAF: Fixed target exepriment, 12 GeV electrons on protons or Lead
- HERA: 27.5 GeV electrons on protons

First we load the relevant modules

In [1]:
import itertools
import sys, string, os
from math import log10, floor

### Global variables
Define some global variables for the production, e.g. working directory, particle id's and associated names, units used for decoding the log files, etc.

*Note*: It is assumed that home_dir contains eSTARlight and the compiled executable e_starlight

In [3]:
home_dir = '/Users/michaellomnitz/Documents/install_test/'
particle_id = [ 113, 333, 443011, 444011, 553011]
#particle_id = [ 444011, 553011]
particle_name = ['rho', 'phi', 'J_psi', 'Psi_2s', 'Upsilon_1s']
#particle_name = [ 'Psi_2s', 'Upsilon_1s']
unit_decode = { 'mb.':-3, 'microb.':-6, 'nanob.':-9, 'picob.':-12, 'femtob.':-15}
unit_latex = { 'mb.':'mb', 'microb.':'$\mu$b', 'nanob.':'nb', 'picob.':'pb', 'femtob.':'fb'}
event_unit = { 3:'K', 6:'M', 9:'G'}
n_events_from_units = {'mb.':'T', 'microb.':'G', 'nanob.':'M', 'picob.':'K', 'femtob.':''} #considering 1 fb^-1

### Function definitions
Some simple funcitons used for utilies and run workflow:
- round_sig: Round a number to a given number of significant figures
- make_input_file: Takes template input file for _eSTARlight_ and sets the desired $Q^2$ range and target nucleii.
- run_accel_study: Iterates over particle species for both photo-($Q^2$ < 1 \rm{GeV}$) and electro-(1< $Q^2$ < 100 \rm{GeV}$ and calls _eSTARlight_. The log file is then saved to desired location
- decode_string: Takes relevant line from _eSTARlight_ and extract the cross-section and units (i.e. reads "gamma+X --> VM+X 204.161 microb." and returns $204.161\times10^{-6}$)
- find_x_section: Iterates over produced log files and extracts cross-sections
- make_table_line: Prints the table as would be formatted for a paper table table

In [13]:

def round_sig(x, sig=2):
    if x < 1e-6:
        return 0
    return round(x, sig-int(floor(log10(abs(x))))-1)

def make_input_file(prod_no,prod_mech, template_loc):
    #load template
    with open(template_loc,'r') as f:
        data = f.readlines()
    # --  Set prodiction id
    data[29] = 'PROD_PID = '+str(particle_id[prod_no])+'   #Channel of interest; this is '+str(particle_name[prod_no])+'\n'
    if prod_mech is 'PP':
        data[33] = 'MIN_GAMMA_Q2 = 0. \n'
        data[34] = 'MAX_GAMMA_Q2 = 1.0 \n'
    if prod_mech is 'EP':
        data[33] = 'MIN_GAMMA_Q2 = 1.0 \n'
        data[34] = 'MAX_GAMMA_Q2 = 100.0 \n'
    with open(home_dir+'slight.in','w') as f:
        f.writelines(data)

def run_accel_study(accel_name, pwd_to_template):
    print(len(particle_id),' particles in study')
    output_dir = home_dir+'eSTARlight/production/accel_x_secs/'+accel_name
    if os.path.isdir(output_dir) is False:
        os.makedirs(output_dir)
        os.makedirs(output_dir+'/PP')
        os.makedirs(output_dir+'/EP')
    for idx,part in enumerate(particle_id):
        make_input_file(idx,'PP',pwd_to_template)
        os.chdir( home_dir )
        os.system( home_dir+'e_starlight > log' )
        os.rename(home_dir+'log',output_dir+'/PP/x_section_'+particle_name[idx]+'.txt')
        #
        #print('Finished with ',particle_name[idx],' photo-production')
        make_input_file(idx,'EP',pwd_to_template)
        os.chdir( home_dir )
        os.system( home_dir+'e_starlight > log' )
        os.rename(home_dir+'log',output_dir+'/EP/x_section_'+particle_name[idx]+'.txt')
    
def decode_string( line ):
    space_pos = [pos for pos,char in enumerate(line) if char == ' ']
    n_space = len(space_pos)
    #print(line[space_pos[n_space-2]:space_pos[n_space-1]])
    num = float(line[space_pos[n_space-2]:space_pos[n_space-1]])
    num = round_sig(num,2)
    units = line[space_pos[n_space-1]+1:-1]
    return num,units
    
def find_x_section(accel_name, prod_mech, part_id):
    file_loc = home_dir+'eSTARlight/production/accel_x_secs/'
    file_loc = file_loc+accel_name+'/'+prod_mech+'/x_section_'+particle_name[part_id]+'.txt'
    with open(file_loc,'r') as f:
        data = f.readlines()
    nline = 0
    for i in range(len(data)):
        if "Total cross section" in data[i]:
            nline = i
    x_sec, units = decode_string(data[nline])
    return x_sec, units

def make_table_line(collision_system, prod_mech, target_A):
    particle_row = ''
    x_sec_row = ' & '
    rate_row = ''
    for idx, a in enumerate(particle_name):
        x_sec, units = find_x_section(collision_system, prod_mech, idx)
        n_events = round(10*x_sec/target_A,2)
        particle_row+=a+' & '
        x_sec_row+='$'+str(x_sec)+'$ '+unit_latex[units]+' & '
        rate_row+= str(n_events)+n_events_from_units[units]
        if idx != len(particle_name)-1:
            rate_row+=' & '
        else:
            rate_row+='\\\\'
    return x_sec_row+rate_row

### Run the study

In [14]:
detectors = ['JLEIC', 'eRHIC', 'LHeC','JLEIC_eA', 'eRHIC_eA', 'LHeC_eA']
for det in detectors:
    temlate_loc = home_dir+'eSTARlight/production/templates/slight_template_'+det+'.in'
    print(' ---- ',det,' ---- ', temlate_loc)
    run_accel_study(det,temlate_loc)


 ----  JLEIC  ----  /Users/michaellomnitz/Documents/install_test/eSTARlight/production/templates/slight_template_JLEIC.in
5  particles in study
 ----  eRHIC  ----  /Users/michaellomnitz/Documents/install_test/eSTARlight/production/templates/slight_template_eRHIC.in
5  particles in study
 ----  LHeC  ----  /Users/michaellomnitz/Documents/install_test/eSTARlight/production/templates/slight_template_LHeC.in
5  particles in study
 ----  JLEIC_eA  ----  /Users/michaellomnitz/Documents/install_test/eSTARlight/production/templates/slight_template_JLEIC_eA.in
5  particles in study
 ----  eRHIC_eA  ----  /Users/michaellomnitz/Documents/install_test/eSTARlight/production/templates/slight_template_eRHIC_eA.in
5  particles in study


FileNotFoundError: [Errno 2] No such file or directory: '/Users/michaellomnitz/Documents/install_test/log' -> '/Users/michaellomnitz/Documents/install_test/eSTARlight/production/accel_x_secseRHIC_eA/EP/x_section_Psi_2s.txt'

## Make paper tables
Using the previously defined tables we can now pepare the tables for the paper. Define the individual rows with their associated rows and the correct scaling for the heavy-ion targets.  The function _make_full_table_ builds the table out of the previous functions. 

In [ ]:
_prod_rows = { 'eRHIC':['eRHIC - $ep$',1], 'eRHIC_eA':['eRHIC - $eA$',197],
               'JLEIC':['JLEIC - $ep$',1], 'JLEIC_eA':['JLEIC - $eA$',208],
               'LHeC' :['LHeC - $ep$',1],  'LHEC_eA' :['LHeC - $eA$',208],
             }
def make_full_table( prod_mech ):
    for idx,key in enumerate(_prod_rows):
        #print(idx,_prod_rows[key][0])
        to_print = _prod_rows[key][0] + make_table_line(key,prod_mech,_prod_rows[key][1])
        print( to_print )
        if (idx+1)%2 == 0:
            print('\hline')
make_full_table('PP') # > 1 GeV

In [7]:
make_full_table('EP') # < 1 GeV

eRHIC - $ep$ & $14.0$ nb & $1.7$ nb & $570.0$ pb & $120.0$ pb & $2.4$ pb & 140.0M & 17.0M & 5700.0K & 1200.0K & 24.0K\\
eRHIC - $eA$ & $730.0$ nb & $110.0$ nb & $77.0$ nb & $19.0$ nb & $200.0$ pb & 37.06M & 5.58M & 3.91M & 0.96M & 10.15K\\
\hline
JLEIC - $ep$ & $10.0$ nb & $1.2$ nb & $270.0$ pb & $55.0$ pb & $790.0$ fb & 100.0M & 12.0M & 2700.0K & 550.0K & 7900.0\\
JLEIC - $eA$ & $450.0$ nb & $67.0$ nb & $25.0$ nb & $5.1$ nb & $0$ fb & 21.63M & 3.22M & 1.2M & 0.25M & 0.0\\
\hline
LHeC - $ep$ & $26.0$ nb & $3.7$ nb & $2.9$ nb & $630.0$ pb & $18.0$ pb & 260.0M & 37.0M & 29.0M & 6300.0K & 180.0K\\
LHeC - $eA$ & $2.0$ $\mu$b & $340.0$ nb & $560.0$ nb & $150.0$ nb & $5.3$ nb & 0.1G & 16.35M & 26.92M & 7.21M & 0.25M\\
\hline
CEBAF - $ep$ & $1.1$ nb & $64.0$ pb & $0$ fb & $0$ fb & $0$ fb & 11.0M & 640.0K & 0.0 & 0.0 & 0.0\\
CEBAF - $ep$ & $0$ fb & $0$ fb & $0$ fb & $0$ fb & $0$ fb & 0.0 & 0.0 & 0.0 & 0.0 & 0.0\\
\hline
HERA - $ep$ & $44.0$ nb & $6.4$ nb & $17.0$ nb & $3.6$ nb & $120.0$ pb & 4